# Module 12 Challenge

### Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Set up Splinter
browser = Browser('chrome')

#### Step 1: Visit the news site
Use automated browser to visit the Mars temperature data site and inspect the page to identify which elements to scrape. 

In [ ]:
# Visit the weather website
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

#### Step 2: Scrape the table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

In [ ]:
# Parse the website
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Find the table